In [118]:
import sys

from iopath.common.file_io import PathManager
from iopath.fb.manifold import ManifoldPathHandler
import torch
import matplotlib.pyplot as plt
sys.path.insert(0, "fbcode/ar_sharing/body_tracking_3d/models/hmr/")

pathmgr = PathManager()
pathmgr.register_handler(ManifoldPathHandler(), allow_override=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
import src

In [132]:
device = "cuda:0"

from src.models.seq_optim import SeqModel

seqOpt = SeqModel(
    "manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth",
    seq_num_frames=339,
)

from src.datasets.datasets_common import UNNORMALIZE

img = UNNORMALIZE(seqOpt.orig_seq["img"])
print(img.shape)

from src.models.raft import raft_pretrained

optical_flow_model = raft_pretrained().to(device)
optical_flow_model = optical_flow_model.eval()

img1 = img[:-1].clone()
img2 = img[1:].clone()

I0920 074229.243 manifold.py:1435] URL manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth


I0920 074229.957 manifold.py:1435] URL manifold://xr_body/tree/personal/andreydavydov/raft/models/raft-things.pth was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/raft/models/raft-things.pth


torch.Size([339, 3, 224, 224])


In [133]:
max_batch = 32
flows_forward_time = []
flows_backward_time = []
with torch.no_grad():
    for img1_, img2_ in zip(torch.split(img1, max_batch), torch.split(img2, max_batch)):
        _, flow_up = optical_flow_model(img1_.to(device), img2_.to(device), iters=20, test_mode=True)
        flows_forward_time.append(flow_up.cpu())
        _, flow_up = optical_flow_model(img2_.to(device), img1_.to(device), iters=20, test_mode=True)
        flows_backward_time.append(flow_up.cpu())

flows_forward_time = torch.cat(flows_forward_time)
flows_backward_time = torch.cat(flows_backward_time)
print(flows_forward_time.shape, flows_backward_time.shape)

torch.Size([338, 2, 224, 224]) torch.Size([338, 2, 224, 224])


In [135]:
t = dict(forward_time=flows_forward_time, backward_time=flows_backward_time)

savepath = "/tmp/optical_flow.pth"
remote_path = "manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/optical_flow_presaved/optical_flows_for_seq.pth"

torch.save(t, savepath)
pathmgr.copy_from_local(savepath, remote_path, overwrite=True)

True